In [1]:
from buttermilk import BM
import os
from hydra import initialize, compose
from omegaconf import OmegaConf


from promptflow.tracing import trace, start_trace
from rich import print as rprint

os.chdir('../..')
print(os.getcwd())

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+flows=summarise_osb","job=export",])



# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)




/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/suzor/src/buttermilk
2024-11-12 13:56:26 J5HW6L4KT6 buttermilk buttermilk.py[ 232] INFO {'message': "Logging setup for: 20241112T0356Z-ensQ-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'export', 'task_id': '20241112T0356Z-ensQ-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/export/20241112T0356Z-ensQ-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'export', 'run_id': '20241112T0356Z-ensQ-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/export/20241112T0356Z-ensQ-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-12 13:56:26 J5HW6L4KT6 buttermilk buttermilk.py[ 240] DEBUG Buttermilk version is: 0.2.0
Prompt flow service has started...


{
    'name': 'automod',
    'job': 'export',
    'logger': {'type': 'gcp'},
    'tracing': {'enabled': True},
    'verbose': True,
    'save_dest': {'type': 'gcp', 'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'secret_provider': {
        'type': 'azure',
        'vault': 'https://suzorvault.vault.azure.net/',
        'resource_group': 'rg-suzor_ai',
        'models_secret': 'models'
    },
    'run': {'platform': 'local', 'max_concurrency': 32},
    'flows': [
        {
            'name': 'summarise OSB cases',
            'num_runs': 1,
            'concurrency': 20,
            'agent': {
                'type': 'LC',
                'template': 'summarise',
                'instructions': 'summarise_osb',
                'model': ['gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'record': 'record'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'plaintext',
                    'uri': 'gs://dmrc-platforms/data/osb/',
                    'glob': '*.md',
                    'columns': {'record_id': 'stem', 'text': 'text'}
                }
            ],
            'save': {
                'type': 'bq',
                'dataset': 'dmrc-analysis.toxicity.flow',
                'schema': 'buttermilk/schemas/flow.json'
            }
        },
        {
            'name': 'synthesise OSB cases',
            'num_runs': 1,
            'concurrency': 20,
            'agent': {
                'type': 'LC',
                'template': 'synthesise',
                'instructions': 'summarise_osb',
                'model': ['gpt4o', 'sonnet']
            },
            'parameters': {'content': 'text', 'answers': 'answers'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'plaintext',
                    'uri': 'gs://dmrc-platforms/data/osb/',
                    'glob': '*.md',
                    'columns': {'record_id': 'stem', 'text': 'text'}
                },
                {
                    'name': 'synthesise previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 2,
                    'agg': True,
                    'filter': {
                        'agent_info.flow': 'summarise OSB cases',
                        'parameters.model': ['gpt4o', 'sonnet', 'gemini15pro']
                    },
                    'join': {'record_id': 'record.record_id'},
                    'group': {
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {'id': 'job_id', 'text': 'outputs'},
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'agent_info': 'agent_info',
                            'model': 'parameters.model',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            ],
            'save': {
                'type': 'bq',
                'dataset': 'dmrc-analysis.toxicity.flow',
                'schema': 'buttermilk/schemas/flow.json'
            }
        },
        {
            'name': 'export',
            'agent': {'type': 'GSheetExporter', 'sheet_name': 'OSB summaries'},
            'save': {'type': 'gsheets', 'destination': 'new'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'plaintext',
                    'uri': 'gs://dmrc-platforms/data/osb/',
                    'glob': '*.md',
                    'columns': {'record_id': 'stem', 'text': 'text'}
     

I0000 00:00:1731383786.993424 62927399 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-12 13:56:27 J5HW6L4KT6 buttermilk save.py[ 272] DEBUG Uploading file gs://dmrc-analysis/runs/automod/export/20241112T0356Z-ensQ-J5HW6L4KT6-suzor/config_fEpFaK.json.


DEBUG:buttermilk:Uploading file gs://dmrc-analysis/runs/automod/export/20241112T0356Z-ensQ-J5HW6L4KT6-suzor/config_fEpFaK.json.


2024-11-12 13:56:28 J5HW6L4KT6 buttermilk save.py[ 275] DEBUG Successfully uploaded file gs://dmrc-analysis/runs/automod/export/20241112T0356Z-ensQ-J5HW6L4KT6-suzor/config_fEpFaK.json with 6256 lines written.


DEBUG:buttermilk:Successfully uploaded file gs://dmrc-analysis/runs/automod/export/20241112T0356Z-ensQ-J5HW6L4KT6-suzor/config_fEpFaK.json with 6256 lines written.


In [2]:
from buttermilk.runner.helpers import load_data
from buttermilk.utils.gsheet import GSheet

from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(flow=cfg.flows[2], source="notebook")

await orchestrator.prepare_data()

df = orchestrator._dataset
df


2024-11-12 13:56:30 J5HW6L4KT6 buttermilk helpers.py[ 216] DEBUG Reading BUN-8S1H6EU5.md from gs://dmrc-platforms/data/osb...


DEBUG:buttermilk:Reading BUN-8S1H6EU5.md from gs://dmrc-platforms/data/osb...


2024-11-12 13:56:34 J5HW6L4KT6 buttermilk helpers.py[ 216] DEBUG Reading BUN-IH313ZHJ.md from gs://dmrc-platforms/data/osb...


DEBUG:buttermilk:Reading BUN-IH313ZHJ.md from gs://dmrc-platforms/data/osb...


2024-11-12 13:56:37 J5HW6L4KT6 buttermilk helpers.py[ 216] DEBUG Reading BUN-QBBLZ8WI.md from gs://dmrc-platforms/data/osb...


...

2024-11-12 13:58:55 J5HW6L4KT6 buttermilk helpers.py[ 216] DEBUG Reading PAO-2021-01.md from gs://dmrc-platforms/data/osb...


DEBUG:buttermilk:Reading PAO-2021-01.md from gs://dmrc-platforms/data/osb...


2024-11-12 13:59:02 J5HW6L4KT6 buttermilk buttermilk.py[ 312] INFO Query stats: Ran in 0:00:02.818682 seconds, cache hit: None, billed None, approx cost $unknown.


INFO:buttermilk:Query stats: Ran in 0:00:02.818682 seconds, cache hit: None, billed None, approx cost $unknown.


,record_id,text,model,timestamp,title,date,content,reasons,recommendations,result,type,topics,standards,location,outputs
14,FB-7UK5F6VG,"{""result"": ""upheld"", ""title"": ""Karachi Mayoral...",sonnet,2024-11-12 02:07:27.221212+00:00,Karachi Mayoral Election Comment,2023-06-01,A user commented on a photo of Karachi politic...,[The Dangerous Organizations and Individuals p...,[Evaluate automated moderation processes to re...,leave up,summary,"[elections, freedom of expression, politics, j...",[Dangerous Organizations and Individuals Policy],Pakistan,{'content': 'A user commented on a photo of Ka...
8,FB-515JVE4X,"{""result"": ""upheld"", ""title"": ""Communal Violen...",sonnet,2024-11-12 02:07:25.870706+00:00,Communal Violence in Indian State of Odisha,2023-04-13,A Facebook user posted a video showing a relig...,[The Violence and Incitement Community Standar...,[Ensure the Violence and Incitement Community ...,takedown,standard,"[violence, religion, communal violence, hate s...",[Violence and Incitement Community Standard],India,{'content': 'A Facebook user posted a video sh...
75,PAO-2021-01,Published\n\n# Sharing private residential inf...,gpt4o,2024-11-12 02:07:16.183222+00:00,Sharing private residential information,2023-10-20,The Board issued a policy advisory opinion rec...,[Under the current Privacy Violations Communit...,[Remove the exception for sharing private resi...,policy advisory,policy advisory,"[Journalism, Marginalized communities, Privacy...",[Privacy Violations Community Standard],Global,{'content': 'The Board issued a policy advisor...
9,FB-659EAWI8,"{""result"": ""upheld"", ""title"": ""Brazilian gener...",gpt4o,2024-11-12 02:06:59.861986+00:00,Brazilian general's speech,2023-01-03,A Facebook video featured a Brazilian general ...,[Tier 3 of the Violence and Incitement Communi...,[Develop a framework for evaluating election i...,takedown,standard,"[elections, governments, protests, incitement,...",[Violence and Incitement Community Standard],Brazil,{'content': 'A Facebook video featured a Brazi...
24,FB-HFFVZENH,"{""result"": ""upheld"", ""title"": ""Girls' Educatio...",sonnet,2024-11-12 02:08:05.725226+00:00,Girls' Education in Afghanistan,2023-07,A user posted text in Pashto discussing the im...,[The Dangerous Organizations and Individuals p...,[Add criteria and illustrative examples to the...,leave up,summary,"[Children's rights, Education, Women's rights,...",[Dangerous Organizations and Individuals],Afghanistan,{'content': 'A user posted text in Pashto disc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,FB-691QAMHJ,"{""result"": ""upheld"", ""title"": ""Former Presiden...",gpt4o,2024-11-12 02:06:57.050479+00:00,Former President Trump's suspension,2021-01-07,"On January 6, 2021, during the counting of the...",[The Dangerous Individuals and Organizations p...,"[Develop clear, necessary, and proportionate p...",takedown,standard,"[Freedom of expression, Politics, Safety]",[Dangerous Individuals and Organizations Commu...,United States,"{'content': 'On January 6, 2021, during the co..."
60,IG-2PJ00L4T,"{""result"": ""upheld"", ""title"": ""Reclaiming Arab...",gpt4o,2024-11-12 02:07:19.213830+00:00,Reclaiming Arabic words,2021-11-17,An Instagram post featured images of Arabic wo...,[Tier 3 of the Hate Speech Community Standard ...,[Translate Meta's Internal Implementation Stan...,leave up,standard,"[LGBT, Marginalized communities, Sex and gende...",[Hate Speech Community Standard],"Egypt, Lebanon, Morocco",{'content': 'An Instagram post featured images...
39,FB-P93JPX02,"{""result"": ""upheld"", ""title"": ""Shared Al Jazee...",gpt4o,2024-11-12 02:06:56.061048+00:00,Shared Al Jazeera post,2021-05-10,"A user in Egypt with over 15,000 followers sha...",[The Dangerous Individuals and Organizations p...,[Add criteria and illustrative examples to the...,leave up,standard,"[Journalism, News reporting, Armed conflict, T...",[Dangerous Individuals and Organizations],Egypt,"{'content': 'A user

In [3]:
# save the input data from this step to a spreadsheet so that we can compare later.
from buttermilk.utils.gsheet import GSheet, format_strings

answers = format_strings(orchestrator._dataset, convert_json_columns=['outputs'])
answers.info()



<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, 14 to 21
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   record_id        154 non-null    object             
 1   text             154 non-null    object             
 2   model            154 non-null    object             
 3   timestamp        154 non-null    datetime64[us, UTC]
 4   title            154 non-null    object             
 5   date             152 non-null    object             
 6   content          154 non-null    object             
 7   reasons          154 non-null    object             
 8   recommendations  154 non-null    object             
 9   result           154 non-null    object             
 10  type             154 non-null    object             
 11  topics           154 non-null    object             
 12  standards        154 non-null    object             
 13  location         154 non-

In [4]:

gsheet = GSheet()
sheet = gsheet.save_gsheet(df=answers,  sheet_name="syntheses", title="OSB Summaries")
sheet_id = sheet.id


2024-11-12 13:59:18 J5HW6L4KT6 buttermilk gsheet.py[ 118] INFO Saved to https://docs.google.com/spreadsheets/d/1bt2PZwMK8GfQJUnquGaHbzOnzkCmMsDMqUJnR0502-c


INFO:buttermilk:Saved to https://docs.google.com/spreadsheets/d/1bt2PZwMK8GfQJUnquGaHbzOnzkCmMsDMqUJnR0502-c


In [5]:
import pandas as pd
import json
orchestrator._dataset.output_backup
# evals = prepare_step_data(cfg.data, cfg.step[3].data)
# evals = evals.set_index(["record_id","model"]).sort_values("timestamp", ascending=False).reset_index().drop_duplicates(["record_id", "model"])
# evals = format_strings(evals, convert_json_columns=['analysis'])
# evals = evals.set_index(["record_id"]).pivot(columns="model",values="analysis")
# sheet = gsheet.save_gsheet(df=evals, sheet_id=sheet_id, sheet_name="evals", title="results")



AttributeError: 'DataFrame' object has no attribute 'output_backup'

In [ ]:
# save the cases df to GCS
# df[['record_id', 'text']].drop_duplicates().reset_index(drop=True).to_json(orient='records', lines=True, path_or_buf='gs://dmrc-platforms/data/osb/cases.json')
